In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/My Drive/EE541_Project"

/content/drive/My Drive/EE541_Project


In [ ]:
!wget https://goo.gl/8hY5ER

--2023-12-04 20:34:18--  https://goo.gl/8hY5ER
Resolving goo.gl (goo.gl)... 108.177.127.139, 108.177.127.100, 108.177.127.113, ...
Connecting to goo.gl (goo.gl)|108.177.127.139|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz [following]
--2023-12-04 20:34:18--  https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz
Resolving zenodo.org (zenodo.org)... 188.185.79.172, 188.184.98.238, 188.184.103.159, ...
Connecting to zenodo.org (zenodo.org)|188.185.79.172|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1203745/files/UrbanSound8K.tar.gz [following]
--2023-12-04 20:34:18--  https://zenodo.org/records/1203745/files/UrbanSound8K.tar.gz
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 6023741708 (5.6G) [application/octet-stream]
Saving to: ‘8hY5ER’

8hY5ER              100%[===================

In [4]:
!tar -xvf "8hY5ER"

串流輸出內容已截斷至最後 5000 行。
UrbanSound8K/audio/fold4/17480-2-0-6.wav
UrbanSound8K/audio/fold4/17480-2-0-9.wav
UrbanSound8K/audio/fold4/175904-2-0-11.wav
UrbanSound8K/audio/fold4/175904-2-0-24.wav
UrbanSound8K/audio/fold4/176003-1-0-0.wav
UrbanSound8K/audio/fold4/176638-5-0-0.wav
UrbanSound8K/audio/fold4/177756-2-0-10.wav
UrbanSound8K/audio/fold4/177756-2-0-4.wav
UrbanSound8K/audio/fold4/177756-2-0-5.wav
UrbanSound8K/audio/fold4/177756-2-0-7.wav
UrbanSound8K/audio/fold4/179862-1-0-0.wav
UrbanSound8K/audio/fold4/180977-3-1-1.wav
UrbanSound8K/audio/fold4/180977-3-1-5.wav
UrbanSound8K/audio/fold4/183989-3-1-21.wav
UrbanSound8K/audio/fold4/183989-3-1-23.wav
UrbanSound8K/audio/fold4/185709-0-0-0.wav
UrbanSound8K/audio/fold4/185709-0-0-1.wav
UrbanSound8K/audio/fold4/185709-0-0-6.wav
UrbanSound8K/audio/fold4/185709-0-0-7.wav
UrbanSound8K/audio/fold4/185909-2-0-102.wav
UrbanSound8K/audio/fold4/185909-2-0-13.wav
UrbanSound8K/audio/fold4/185909-2-0-17.wav
UrbanSound8K/audio/fold4/185909-2-0-34.wav
Urban

In [13]:
!ls

8hY5ER	DataCollection.py  Data_Processing.ipynb  __pycache__  UrbanSound8K


In [3]:
import progressbar
import time
import os
import struct
import matplotlib.pyplot as plt
import IPython.display as ipd
import pandas as pd
import numpy as np
import librosa # for sound processing.
import DataCollection as dc # a local module

In [4]:
data = pd.read_csv("/content/drive/My Drive/EE541_Project/UrbanSound8K/metadata/UrbanSound8K.csv")
data.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [7]:
data.shape[0]

8732

In [8]:
###data = data.sort_values(by=['fold', 'classID',"fsID"], ascending=[True, True, True])
dataset = np.zeros(shape = (data.shape[0],2),dtype = object)
dataset.shape

(8732, 2)

In [10]:
bar = progressbar.ProgressBar(maxval=data.shape[0], widgets=[progressbar.Bar('$', '||', '||'), ' ', progressbar.Percentage()])
bar.start()
for i in range(data.shape[0]):

    fullpath, class_id = dc.path_class(data,data.slice_file_name[i])
    try:
        X, sample_rate = librosa.load(fullpath)
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    except Exception:
        print("Error encountered while parsing file: ")
        mfccs,class_id = None, None
    feature = mfccs
    label = class_id
    dataset[i,0],dataset[i,1] = feature,label

    bar.update(i+1)

||$$$$$$$$$$$$$$$$$$$$$$$$$$$$                                          ||  40%/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1323
  warnings.warn(
||$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$    ||  94%/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1523
  warnings.warn(
||$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$|| 100%

In [11]:
np.save("/content/drive/My Drive/EE541_Project/dataset_melspectrogram",dataset)

In [12]:
dataset[0]

array([array([-2.11936981e+02,  6.25812149e+01, -1.22813156e+02, -6.07452927e+01,
              -1.38937550e+01, -2.97898350e+01, -3.97887087e+00,  1.16827412e+01,
               1.29638319e+01,  8.33642006e+00, -6.52981663e+00, -2.99610305e+00,
              -1.28879461e+01,  5.88593531e+00, -2.47522211e+00, -4.40830946e+00,
              -1.50144749e+01, -9.43993688e-01, -2.57990861e+00,  6.09830046e+00,
              -1.07862263e+01, -8.06347752e+00,  5.84409714e+00, -3.12253737e+00,
              -5.98421907e+00, -7.99421892e-02, -3.68708348e+00,  4.74768400e+00,
              -6.53008938e+00, -5.08387709e+00,  4.87733096e-01,  7.51088953e+00,
              -8.85352790e-01,  2.39381790e+00, -6.39236736e+00, -2.13185430e+00,
               2.27601957e+00, -7.91079581e-01, -1.54006314e+00,  1.32115340e+00],
             dtype=float32)                                                       ,
       'dog_bark'], dtype=object)

In [13]:
l = np.load("dataset_melspectrogram.npy", allow_pickle=True)

In [14]:
l.shape

(8732, 2)

In [15]:
l[8730,1]

'car_horn'

In [16]:
l[8730, 0]

array([-3.36923859e+02,  1.15313904e+02, -4.48470917e+01,  2.49689846e+01,
       -9.88802528e+00, -2.15468502e+00,  2.27000904e+01,  2.34235898e-01,
       -1.22908525e-01,  1.01740675e+01, -2.78273892e+00, -6.99572229e+00,
       -4.81200695e-01,  5.91099787e+00,  1.23319614e+00,  9.75832272e+00,
        3.33471704e+00,  1.92964590e+00, -1.89814579e+00,  1.15353441e+00,
       -5.89131832e+00,  1.67433333e+00, -9.03743458e+00,  3.31660795e+00,
        2.03650665e+00,  2.32190084e+00, -4.17932558e+00,  3.91128492e+00,
        1.53706753e+00, -2.00531149e+00, -8.69075012e+00,  2.76224756e+00,
        5.43595886e+00,  6.60244846e+00,  7.74977088e-01,  3.78668571e+00,
       -5.38626909e+00, -6.76911306e+00, -2.85838604e+00,  6.95349121e+00],
      dtype=float32)